In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from skimage.feature import local_binary_pattern
import mahotas
import glob

# === Local Paths ===
skin_type_train_dir = # training path of dataset
skin_type_valid_dir = # validation path of dataset 

# === Feature Extraction ===
def extract_skin_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img, (128, 128))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Color Histograms (RGB + HSV)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    color_feat = np.concatenate([
        cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0,256,0,256,0,256]).flatten(),
        cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0,180,0,256,0,256]).flatten()
    ])

    # Texture - LBP
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
    lbp_hist = lbp_hist.astype("float") / (lbp_hist.sum() + 1e-7)

    # Texture - Haralick
    haralick_feat = mahotas.features.haralick(gray).mean(axis=0)

    return np.concatenate([color_feat, lbp_hist, haralick_feat])

# === Dataset Loader ===
def load_dataset(path, classes):
    X, y = [], []
    for i, label in enumerate(classes):
        folder = os.path.join(path, label)
        files = glob.glob(folder + '/*.jpg') + glob.glob(folder + '/*.png')
        for file in files:
            features = extract_skin_features(file)
            if features is not None:
                X.append(features)
                y.append(i)
    return np.array(X), np.array(y)

# === Load Skin Type Dataset ===
skin_classes = ['dry', 'normal', 'oily']
X_train_skin, y_train_skin = load_dataset(skin_type_train_dir, skin_classes)
X_val_skin, y_val_skin = load_dataset(skin_type_valid_dir, skin_classes)

# === Combine and Shuffle ===
X_skin, y_skin = shuffle(np.concatenate([X_train_skin, X_val_skin]),
                         np.concatenate([y_train_skin, y_val_skin]),
                         random_state=42)

# === SMOTE Balancing ===
smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X_skin, y_skin)

# === Scaling + PCA ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_bal)

# Reduce dimensionality to improve speed + accuracy
pca = PCA(n_components=100, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_bal, test_size=0.2,
                                                    stratify=y_bal, random_state=42)

# === SVM with Grid Search ===
param_grid = {'svc__C': [1, 10], 'svc__gamma': ['scale', 0.01]}
svm_model = GridSearchCV(make_pipeline(StandardScaler(), SVC(kernel='rbf')), 
                         param_grid=param_grid, cv=3, n_jobs=-1)
svm_model.fit(X_train, y_train)

# === Evaluate ===
y_pred = svm_model.predict(X_test)
print("✅ Skin Type Classification Accuracy:", accuracy_score(y_test, y_pred))
print("\n🧾 Classification Report:")
print(classification_report(y_test, y_pred, target_names=skin_classes))


import joblib

# Save model components
joblib.dump(svm_model, "skin_type_svm_model.pkl")
joblib.dump(pca, "skin_type_pca.pkl")
joblib.dump(scaler, "skin_type_scaler.pkl")

✅ Skin Type Classification Accuracy: 0.6707818930041153

🧾 Classification Report:
              precision    recall  f1-score   support

         dry       0.70      0.70      0.70       243
      normal       0.65      0.59      0.62       243
        oily       0.66      0.72      0.69       243

    accuracy                           0.67       729
   macro avg       0.67      0.67      0.67       729
weighted avg       0.67      0.67      0.67       729



['skin_type_scaler.pkl']

In [18]:
import joblib
import cv2
import numpy as np
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from skimage.feature import local_binary_pattern
import mahotas
import glob

In [ ]:
def predict_skin_type(image_path):
    # Load model components
    model = joblib.load("models/models/skin_type_svm_model.pkl")
    pca = joblib.load("models/models/skin_type_pca.pkl")
    scaler = joblib.load("models/models/skin_type_scaler.pkl")

    # Class labels
    skin_classes = ['dry', 'normal', 'oily']

    # Extract features
    features = extract_skin_features(image_path)
    if features is None:
        print("Image could not be loaded.")
        return

    # Transform
    features_scaled = scaler.transform([features])
    features_pca = pca.transform(features_scaled)

    # Predict
    prediction = model.predict(features_pca)[0]
    print(f"🧠 Predicted Skin Type: {skin_classes[prediction]}")

predict_skin_type(#sample input image path to predict)


🧠 Predicted Skin Type: oily


In [11]:
def extract_skin_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img, (128, 128))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Color Histograms (RGB + HSV)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    color_feat = np.concatenate([
        cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0,256,0,256,0,256]).flatten(),
        cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0,180,0,256,0,256]).flatten()
    ])

    # Texture - LBP
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
    lbp_hist = lbp_hist.astype("float") / (lbp_hist.sum() + 1e-7)

    # Texture - Haralick
    haralick_feat = mahotas.features.haralick(gray).mean(axis=0)

    return np.concatenate([color_feat, lbp_hist, haralick_feat])
